# Churn Prediction & Notification Uplift Model
**Predicting Churn Timing and Measuring Notification Impact for Long-Tenured Wellness Users**

---

## Project Overview

This notebook implements a two-model approach following the **CRISP-DM** methodology:

1. **Churn Prediction Model** — An XGBoost classifier that detects behavioral drift (declining sessions, growing inactivity gaps, reduced workout duration) to identify users likely to cancel within 30 days.
2. **Uplift Analysis** — Measures the causal impact of push notifications on churn reduction across user segments using real notification data.

### Key Design Decisions
- **Long-tenured users only** — All users have 180+ days of tenure. The model focuses on retention bleed, not early-stage drop-offs.
- **Drift-based features** — Rather than static snapshots, the model compares a user's *baseline* activity (45–90 days before decision) against their *recent* activity (last 30 days) to detect disengagement trajectories.
- **Temporal integrity** — All features are computed from activity strictly before the decision date to prevent data leakage.
- **Separated concerns** — The churn prediction model and the uplift analysis are kept independent; the notification flag is NOT used as a model feature.

## 1. Configuration & Data Loading

The dataset consists of six tables covering user demographics, subscription details, workout sessions, support contacts, notification logs, and cancellation events.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    roc_auc_score, roc_curve, classification_report,
    confusion_matrix, precision_recall_curve, average_precision_score
)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 11

DATA_PATH = 'data/agile_churn_raw_v11.xlsx'

print(f'Loading data from: {DATA_PATH}...')
df_users    = pd.read_excel(DATA_PATH, sheet_name='users_raw')
df_subs     = pd.read_excel(DATA_PATH, sheet_name='subscriptions_raw')
df_cancel   = pd.read_excel(DATA_PATH, sheet_name='cancellations_raw')
df_notif    = pd.read_excel(DATA_PATH, sheet_name='notifications_raw')
df_sessions = pd.read_excel(DATA_PATH, sheet_name='sessions_raw')
df_support  = pd.read_excel(DATA_PATH, sheet_name='support_raw')

print(f'Loaded — {len(df_users):,} users | {len(df_sessions):,} sessions | '
      f'{len(df_cancel):,} cancellations | {len(df_notif):,} notifications')

## 2. Data Cleaning & Type Conversion

In [ ]:
df_users['decision_date']  = pd.to_datetime(df_users['decision_date'])
df_subs['first_paid_date'] = pd.to_datetime(df_subs['first_paid_date'])
df_cancel['churn_ts']      = pd.to_datetime(df_cancel['churn_ts'])
df_sessions['start_ts']    = pd.to_datetime(df_sessions['start_ts'])
df_notif['sent_ts']        = pd.to_datetime(df_notif['sent_ts'])
df_support['contact_ts']   = pd.to_datetime(df_support['contact_ts'])

print('Date conversion complete.')
print(f'User decision dates: {df_users["decision_date"].min().date()} to {df_users["decision_date"].max().date()}')
print(f'Session dates: {df_sessions["start_ts"].min().date()} to {df_sessions["start_ts"].max().date()}')

## 3. Population & Churn Definition

All users in this dataset are long-tenured (180+ day tenure), which aligns with the project scope of preventing retention bleed among established subscribers.

**Churn definition:** A user is labeled as churned if their cancellation event falls within a 30-day forward window from their decision date.

In [ ]:
# Merge base tables
model_df = df_users.merge(df_subs, on='user_id', how='left')
model_df = model_df.merge(df_cancel[['user_id', 'churn_ts']], on='user_id', how='left')

# Tenure
model_df['tenure_days'] = (model_df['decision_date'] - model_df['first_paid_date']).dt.days

# Explicit tenure guardrail
assert model_df['tenure_days'].min() >= 180, (
    f"Expected all users to have 180+ day tenure, but min is {model_df['tenure_days'].min()}"
)
print(f'Population: {len(model_df):,} long-tenured users (all >= 180 days)')
print(f'Tenure range: {model_df["tenure_days"].min()} – {model_df["tenure_days"].max()} days '
      f'(mean: {model_df["tenure_days"].mean():.0f})')

# Churn label (30-day forward window)
CHURN_WINDOW = 30
churn_mask = (
    (model_df['churn_ts'].notna()) &
    (model_df['churn_ts'] >= model_df['decision_date']) &
    (model_df['churn_ts'] <= model_df['decision_date'] + pd.Timedelta(days=CHURN_WINDOW))
)
model_df['target_churn'] = churn_mask.astype(int)

churn_rate = model_df['target_churn'].mean()
print(f'\nChurn rate ({CHURN_WINDOW}-day window): {churn_rate:.1%}')
print(f'  Churners: {model_df["target_churn"].sum():,}  |  Retained: {(model_df["target_churn"]==0).sum():,}')

## 4. Drift-Based Feature Engineering

This is the core of the modeling approach. Instead of using static, point-in-time metrics, we compute **behavioral drift** — the change between a user's established baseline and their recent activity.

### Time Windows
| Window | Period | Purpose |
|--------|--------|---------|
| **Baseline** | 45–90 days before decision | Established behavioral patterns |
| **Recent** | 0–30 days before decision | Current engagement state |

### Drift Features
| Feature | Formula | Signal |
|---------|---------|--------|
| `session_decline_rate` | (recent_count − base_count) / base_count | Frequency decay |
| `duration_drift` | recent_avg_duration − base_avg_duration | Engagement quality drop |
| `inactivity_gap_growth` | recent_avg_gap − base_avg_gap | Growing gaps between sessions |
| `calorie_trend` | (recent_calories − base_calories) / base_calories | Effort decline |
| `days_since_last_workout` | Days from last session to decision date | Immediate recency signal |

In [ ]:
# ── Merge sessions with decision dates ──
sess = df_sessions.merge(df_users[['user_id', 'decision_date']], on='user_id', how='left')
sess['days_before'] = (sess['decision_date'] - sess['start_ts']).dt.days
sess_pre = sess[sess['days_before'] >= 0]  # only pre-decision sessions

# ── Define time windows ──
baseline_window = sess_pre[(sess_pre['days_before'] >= 45) & (sess_pre['days_before'] <= 90)]
recent_window   = sess_pre[sess_pre['days_before'] <= 30]

print(f'Baseline window (45-90 days): {len(baseline_window):,} sessions across {baseline_window["user_id"].nunique():,} users')
print(f'Recent window (0-30 days):    {len(recent_window):,} sessions across {recent_window["user_id"].nunique():,} users')

def window_agg(df, prefix):
    """Aggregate session metrics within a time window."""
    agg = df.groupby('user_id').agg(
        count=('session_id', 'count'),
        avg_dur=('duration_min', 'mean'),
        total_cal=('calories', 'sum'),
    ).reset_index()
    # Average gap between consecutive sessions
    gaps = (df.sort_values(['user_id', 'start_ts'])
            .groupby('user_id')['start_ts']
            .apply(lambda x: x.diff().dt.days.mean() if len(x) > 1 else np.nan)
            .reset_index(name='avg_gap'))
    agg = agg.merge(gaps, on='user_id', how='left')
    return agg.rename(columns={c: f'{prefix}_{c}' for c in agg.columns if c != 'user_id'})

base_features = window_agg(baseline_window, 'base')
recent_features = window_agg(recent_window, 'recent')

# ── Compute drift metrics ──
drift_df = (df_users[['user_id']]
            .merge(base_features, on='user_id', how='left')
            .merge(recent_features, on='user_id', how='left'))

# Fill missing windows (no sessions in that period)
drift_df = drift_df.fillna({
    'base_count': 0, 'recent_count': 0,
    'base_avg_dur': 0, 'recent_avg_dur': 0,
    'base_total_cal': 0, 'recent_total_cal': 0,
    'base_avg_gap': 999, 'recent_avg_gap': 999
})

# Session decline rate
drift_df['session_decline_rate'] = np.where(
    drift_df['base_count'] > 0,
    (drift_df['recent_count'] - drift_df['base_count']) / drift_df['base_count'],
    np.where(drift_df['recent_count'] > 0, 0, -1)
)

# Duration drift (change in avg workout length)
drift_df['duration_drift'] = drift_df['recent_avg_dur'] - drift_df['base_avg_dur']

# Inactivity gap growth (widening gaps = disengagement)
drift_df['inactivity_gap_growth'] = drift_df['recent_avg_gap'] - drift_df['base_avg_gap']

# Calorie trend
drift_df['calorie_trend'] = np.where(
    drift_df['base_total_cal'] > 0,
    (drift_df['recent_total_cal'] - drift_df['base_total_cal']) / drift_df['base_total_cal'],
    0
)

print('\nDrift features computed.')

In [ ]:
# ── Overall session metrics & recency ──
total_stats = sess_pre.groupby('user_id').agg(
    total_sessions=('session_id', 'count'),
    avg_duration=('duration_min', 'mean'),
    total_calories=('calories', 'sum'),
    days_since_last_workout=('days_before', 'min'),
    workout_type_mode=('workout_type', lambda x: x.mode()[0] if not x.mode().empty else 'Other')
).reset_index()

# ── Support tickets ──
support_counts = df_support.groupby('user_id').size().reset_index(name='support_tickets')

# ── Merge all features ──
model_df = model_df.merge(drift_df, on='user_id', how='left')
model_df = model_df.merge(total_stats, on='user_id', how='left')
model_df = model_df.merge(support_counts, on='user_id', how='left')

# ── Imputation ──
model_df['total_sessions'] = model_df['total_sessions'].fillna(0)
model_df['avg_duration'] = model_df['avg_duration'].fillna(0)
model_df['total_calories'] = model_df['total_calories'].fillna(0)
model_df['days_since_last_workout'] = model_df['days_since_last_workout'].fillna(999)
model_df['support_tickets'] = model_df['support_tickets'].fillna(0)
model_df['workout_type_mode'] = model_df['workout_type_mode'].fillna('None')

# ── Categorical encodings ──
model_df['gender_code'] = model_df['gender'].astype('category').cat.codes
model_df['billing_code'] = model_df['billing_cycle'].astype('category').cat.codes
model_df['activity_code'] = model_df['workout_type_mode'].astype('category').cat.codes

# ── Activity cohorts (tercile buckets) ──
model_df['activity_bucket'] = pd.qcut(
    model_df['total_sessions'], q=3,
    labels=['Low Activity', 'Medium Activity', 'High Activity']
)

# ── Notification flag (time-window enforced) ──
# Only count notifications sent 7-30 days BEFORE the decision date
NOTIF_WINDOW_START = 7   # days before decision
NOTIF_WINDOW_END   = 30  # days before decision

notif_timed = df_notif.merge(df_users[['user_id', 'decision_date']], on='user_id', how='left')
notif_timed['days_before'] = (notif_timed['decision_date'] - notif_timed['sent_ts']).dt.days
valid_notif = notif_timed[
    (notif_timed['days_before'] >= NOTIF_WINDOW_START) &
    (notif_timed['days_before'] <= NOTIF_WINDOW_END)
]
notified_users = set(valid_notif['user_id'].unique())
model_df['received_notification'] = model_df['user_id'].isin(notified_users).astype(int)

print(f'Feature engineering complete — {model_df.shape[1]} columns.')
print(f'\nActivity cohort distribution:')
print(model_df['activity_bucket'].value_counts().sort_index())
print(f'\nNotification (time-window enforced, {NOTIF_WINDOW_START}-{NOTIF_WINDOW_END} days pre-decision):')
print(f'  Notified: {model_df["received_notification"].sum():,} ({model_df["received_notification"].mean():.1%})')
print(f'  Control:  {(model_df["received_notification"]==0).sum():,}')

## 5. Exploratory Data Analysis

### 5a. Drift Features vs Churn
If the drift features capture real disengagement signals, churners should show declining sessions, shrinking durations, and growing inactivity gaps compared to retained users.

In [ ]:
# Drift features: churned vs retained
drift_cols = ['session_decline_rate', 'duration_drift', 'inactivity_gap_growth',
              'days_since_last_workout', 'recent_count', 'base_count']

print('Drift Features — Churned vs Retained (mean):')
print('-' * 75)
for col in drift_cols:
    churned = model_df[model_df['target_churn'] == 1][col].mean()
    retained = model_df[model_df['target_churn'] == 0][col].mean()
    print(f'  {col:30s}  Churned: {churned:+9.2f}  |  Retained: {retained:+9.2f}  |  Δ: {churned - retained:+9.2f}')

In [ ]:
# Churn rate by activity cohort
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.barplot(x='activity_bucket', y='target_churn', data=model_df,
            palette='viridis', ci=None, ax=axes[0])
axes[0].set_title('Churn Rate by Activity Cohort')
axes[0].set_ylabel('Churn Probability')

# Drift distributions by churn status
sns.boxplot(x='target_churn', y='days_since_last_workout', data=model_df,
            palette='Set2', ax=axes[1])
axes[1].set_title('Recency: Days Since Last Workout')
axes[1].set_xlabel('Churned')

sns.boxplot(x='target_churn', y='duration_drift', data=model_df,
            palette='Set2', ax=axes[2])
axes[2].set_title('Duration Drift (Recent − Baseline)')
axes[2].set_xlabel('Churned')

plt.tight_layout()
plt.show()

In [ ]:
# Additional EDA: support tickets and session decline
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.boxplot(x='target_churn', y='support_tickets', data=model_df,
            palette='Set2', ax=axes[0])
axes[0].set_title('Friction: Support Tickets')
axes[0].set_xlabel('Churned')

sns.boxplot(x='target_churn', y='session_decline_rate', data=model_df,
            palette='Set2', ax=axes[1])
axes[1].set_title('Session Decline Rate (Recent vs Baseline)')
axes[1].set_xlabel('Churned')

plt.tight_layout()
plt.show()

## 6. Churn Prediction Model (XGBoost)

The model is trained on drift-based features, overall activity metrics, demographics, and friction signals. The `received_notification` flag is deliberately **excluded** — the churn model predicts risk independent of interventions, while notification impact is measured separately in the uplift analysis.

We validate with **5-fold stratified cross-validation** to confirm the model generalizes, then train a final model on an 80/20 holdout for detailed evaluation.

In [ ]:
# ── Feature set (no treatment indicator) ──
FEATURES = [
    # Drift features (core signal)
    'session_decline_rate', 'duration_drift',
    'inactivity_gap_growth', 'calorie_trend',
    'recent_count', 'base_count',
    # Overall metrics
    'total_sessions', 'avg_duration', 'days_since_last_workout',
    # Demographics & friction
    'age', 'height_cm', 'weight_kg', 'tenure_days',
    'support_tickets', 'gender_code', 'billing_code'
]

X = model_df[FEATURES].fillna(0)
y = model_df['target_churn']

# ── Cross-validation ──
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    eval_metric='logloss',
    random_state=42
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(xgb_model, X, y, cv=cv, scoring='roc_auc')

print(f'5-Fold Stratified CV ROC-AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})')
print(f'Per-fold scores: {[f"{s:.4f}" for s in cv_scores]}')

# ── Final holdout model ──
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'\nTraining on {len(X_train):,} samples, evaluating on {len(X_test):,}...')
xgb_model.fit(X_train, y_train)
print('Training complete.')

## 7. Model Evaluation

For imbalanced churn data, ROC-AUC alone is insufficient. We also evaluate with:
- **Precision-Recall curve** — Measures how well the model captures true churners without excessive false alarms
- **Youden's J statistic** — Finds the optimal classification threshold
- **Classification report** — Precision, recall, and F1 at the optimal threshold

In [ ]:
y_probs = xgb_model.predict_proba(X_test)[:, 1]

# ── ROC Curve ──
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
auc_score = roc_auc_score(y_test, y_probs)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].plot(fpr, tpr, label=f'XGBoost (AUC = {auc_score:.3f})', color='purple', lw=2)
axes[0].scatter(fpr[ix], tpr[ix], marker='o', color='black',
                label=f'Optimal Threshold = {best_thresh:.2f}', zorder=5)
axes[0].plot([0, 1], [0, 1], linestyle='--', color='gray')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve')
axes[0].legend()

# ── Precision-Recall Curve ──
precision, recall, _ = precision_recall_curve(y_test, y_probs)
ap_score = average_precision_score(y_test, y_probs)

axes[1].plot(recall, precision, label=f'AP = {ap_score:.2f}', color='teal', lw=2)
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend()

plt.tight_layout()
plt.show()

# ── Classification Report ──
y_pred = (y_probs >= best_thresh).astype(int)
print(f'Classification Report (threshold = {best_thresh:.2f}):\n')
print(classification_report(y_test, y_pred))

## 8. Model Explainability (SHAP)

SHAP values provide transparency into which features drive each prediction. If the model is sound, drift-based features should dominate the importance rankings — confirming the model relies on behavioral change signals rather than static demographics.

In [ ]:
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)

# Bar chart (global importance)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, plot_type='bar', show=False)
plt.title('Feature Importance (SHAP)')
plt.tight_layout()
plt.show()

# Beeswarm (direction of impact)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, show=False)
plt.title('Feature Impact Direction')
plt.tight_layout()
plt.show()

## 9. Uplift Analysis — Notification Impact

This section estimates the effect of push notifications on churn using an **observational uplift** approach with **propensity score adjustment**.

### Design
- **Treatment definition:** Users who received a notification within a specific pre-decision window (7–30 days before decision date). This time-window enforcement ensures the notification plausibly preceded the churn decision.
- **Control group:** Users who did not receive a notification in that window.
- **Confounding adjustment:** Since treatment was not randomly assigned, users who received notifications may differ systematically from those who didn't. We use **Inverse Propensity Weighting (IPW)** to adjust for baseline covariates, producing an estimate closer to causal.

### Limitations
- This is an **observational study**, not a randomized experiment. Unmeasured confounders may still bias the estimate.
- IPW adjusts for *observed* differences only. A true A/B test would be needed to confirm causality.
- Results should be interpreted as **adjusted associations** rather than definitive causal effects.

In [ ]:
from sklearn.linear_model import LogisticRegression

# ── 9a. Naive (unadjusted) uplift ──
print('=' * 65)
print('  NAIVE UPLIFT (unadjusted)')
print('=' * 65)

naive_rates = model_df.groupby('received_notification')['target_churn'].agg(['mean', 'count'])
naive_rates.index = ['Control', 'Treatment']
naive_rates.columns = ['Churn Rate', 'Users']
print(naive_rates)
naive_uplift = naive_rates.loc['Treatment', 'Churn Rate'] - naive_rates.loc['Control', 'Churn Rate']
print(f'Naive uplift: {naive_uplift*100:+.2f} pp\n')

# ── 9b. Propensity Score (IPW) adjusted uplift ──
print('=' * 65)
print('  IPW-ADJUSTED UPLIFT')
print('=' * 65)

# Covariates for propensity model
PS_COVARIATES = ['age', 'tenure_days', 'total_sessions', 'avg_duration',
                 'days_since_last_workout', 'support_tickets', 'gender_code', 'billing_code']

X_ps = model_df[PS_COVARIATES].fillna(0)
T = model_df['received_notification']
Y = model_df['target_churn']

# Fit propensity score model
ps_model = LogisticRegression(max_iter=1000, random_state=42)
ps_model.fit(X_ps, T)
propensity = ps_model.predict_proba(X_ps)[:, 1]

# Clip propensity scores to avoid extreme weights
propensity = np.clip(propensity, 0.05, 0.95)

# IPW weights
weights = np.where(T == 1, 1 / propensity, 1 / (1 - propensity))

# Weighted churn rates
treated_mask = T == 1
control_mask = T == 0

ipw_treatment_rate = np.average(Y[treated_mask], weights=weights[treated_mask])
ipw_control_rate = np.average(Y[control_mask], weights=weights[control_mask])
ipw_uplift = ipw_treatment_rate - ipw_control_rate

print(f'IPW-adjusted Treatment churn rate: {ipw_treatment_rate:.3f}')
print(f'IPW-adjusted Control churn rate:   {ipw_control_rate:.3f}')
print(f'IPW-adjusted uplift: {ipw_uplift*100:+.2f} pp')
print(f'\nPropensity score distribution: mean={propensity.mean():.3f}, '
      f'std={propensity.std():.3f}, range=[{propensity.min():.3f}, {propensity.max():.3f}]')

In [ ]:
# ── 9c. Segment-level uplift (unadjusted, for directional comparison) ──
print('\n' + '=' * 65)
print('  UPLIFT BY SEGMENT (unadjusted — directional only)')
print('=' * 65)

# By activity cohort
cohort_uplift = model_df.groupby(
    ['activity_bucket', 'received_notification']
)['target_churn'].mean().unstack()
cohort_uplift.columns = ['Control', 'Treatment']
cohort_uplift['Uplift_PP'] = (cohort_uplift['Treatment'] - cohort_uplift['Control']) * 100
print('\nBy Activity Cohort:')
print(cohort_uplift)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# By cohort
x = np.arange(len(cohort_uplift))
w = 0.35
axes[0].bar(x - w/2, cohort_uplift['Control'], w, label='Control', color='#EF5350', alpha=0.85)
axes[0].bar(x + w/2, cohort_uplift['Treatment'], w, label='Notified', color='#42A5F5', alpha=0.85)
axes[0].set_xticks(x)
axes[0].set_xticklabels(cohort_uplift.index)
axes[0].set_ylabel('Churn Rate')
axes[0].set_title('Notification Impact by Cohort (Unadjusted)', fontweight='bold')
axes[0].legend()

# Overall comparison: naive vs IPW
bars = axes[1].bar(
    ['Naive\nControl', 'Naive\nTreatment', 'IPW\nControl', 'IPW\nTreatment'],
    [naive_rates.loc['Control','Churn Rate'], naive_rates.loc['Treatment','Churn Rate'],
     ipw_control_rate, ipw_treatment_rate],
    color=['#EF5350','#42A5F5','#EF5350','#42A5F5'],
    alpha=[0.5, 0.5, 1.0, 1.0], width=0.5
)
axes[1].set_ylabel('Churn Rate')
axes[1].set_title('Naive vs IPW-Adjusted Churn Rates', fontweight='bold')
axes[1].axhline(y=model_df['target_churn'].mean(), color='gray', linestyle='--', alpha=0.5, label='Overall rate')
axes[1].legend()

plt.tight_layout()
plt.show()

# By workout type
type_uplift = model_df.groupby(
    ['workout_type_mode', 'received_notification']
)['target_churn'].mean().unstack()
type_uplift.columns = ['Control', 'Treatment']
type_uplift['Uplift_PP'] = (type_uplift['Treatment'] - type_uplift['Control']) * 100
print('\nBy Workout Type:')
print(type_uplift)

## 10. Appendix — Data Distributions & Correlations

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(model_df['age'], bins=15, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Age Distribution')

sns.countplot(x='gender', data=model_df, ax=axes[1], palette='pastel')
axes[1].set_title('Gender Distribution')

plt.tight_layout()
plt.show()

# Correlation matrix
plt.figure(figsize=(10, 8))
corr_cols = ['target_churn', 'days_since_last_workout', 'session_decline_rate',
             'duration_drift', 'total_sessions', 'support_tickets', 'age']
sns.heatmap(model_df[corr_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

## 11. Automated Business Insights

In [ ]:
print('=' * 60)
print('  BUSINESS INSIGHTS SUMMARY')
print('=' * 60)

# 1. Baseline
print(f'1. BASELINE: Overall churn rate is {model_df["target_churn"].mean():.1%}.')

# 2. Risk signal
median_recency = model_df[model_df['target_churn']==1]['days_since_last_workout'].median()
print(f'2. RISK SIGNAL: Churned users stop working out ~{median_recency:.0f} days before cancelling.')

# 3. Drift insight
churned_decline = model_df[model_df['target_churn']==1]['session_decline_rate'].mean()
retained_decline = model_df[model_df['target_churn']==0]['session_decline_rate'].mean()
print(f'3. DRIFT: Churners show session decline of {churned_decline:+.2f} vs {retained_decline:+.2f} for retained.')

# 4. Cohorts
best_cohort = cohort_uplift['Control'].idxmin()
worst_cohort = cohort_uplift['Control'].idxmax()
print(f'4. COHORTS: \'{best_cohort}\' users are safest. \'{worst_cohort}\' users are most at risk.')

# 5. Notification strategy
overall_uplift = (treatment_rate - control_rate) * 100
print(f'5. NOTIFICATIONS: {overall_uplift:+.2f} pp churn reduction for notified users.')

# 6. Model performance
print(f'6. MODEL: ROC-AUC = {auc_score:.3f} | CV-AUC = {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})')

print('=' * 60)